In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV,ParameterGrid,cross_val_score
from sklearn.neural_network import MLPClassifier

In [2]:
# train = pd.read_csv('data_source/train.csv')
# test = pd.read_csv('data_source/test.csv')

# train_robust = train.copy()
# test_robust = test.copy()

x_train = pd.read_csv('data_source/X_train.csv')
y_train = pd.read_csv('data_source/y_train.csv')

x_test = pd.read_csv('data_source/X_test.csv')
y_test = pd.read_csv('data_source/y_test.csv')

x_train_std = x_train.copy()
y_train_std = y_train.copy()

x_train_robust = x_train.copy()
y_train_robust = y_train.copy()

x_test_std = x_test.copy()
y_test_std = y_test.copy()

x_test_robust = x_test.copy()
y_test_robust = y_test.copy()

x_train_std = x_train_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_std = y_train_std.loc[:,'Action']
x_test_std = x_test_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_std = y_test_std.loc[:,'Action']

x_train_robust = x_train_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_robust = y_train_robust.loc[:,'Action']
x_test_robust = x_test_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_robust = y_test_robust.loc[:,'Action']

In [3]:
scaler = StandardScaler()
robust = RobustScaler()

x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])

x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])


In [4]:
clf = MLPClassifier()
clf.fit(x_train_robust,y_train_robust)
answer = clf.predict(x_test_robust)
print(classification_report(y_test_robust, answer))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93     11292
           1       0.86      0.17      0.28      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.94      0.00        16

    accuracy                           0.54     19660
   macro avg       0.47      0.50      0.31     19660
weighted avg       0.77      0.54      0.60     19660



/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [5]:
parameters = [
    {'activation':['identity','logistic','tanh','relu'],'learning_rate':['constant','invscaling','adaptive']} # ,'solver':['lbfgs','sgd','adam']
]

### Parameter Grid for Robust

In [6]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_robust,y_train_robust)
    answer = clf.predict(x_test_robust)
    f1 = f1_score(y_test_robust,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


F1: 0.72935
Grid: {'activation': 'tanh', 'learning_rate': 'constant'}


In [9]:
clf = MLPClassifier(activation='tanh',learning_rate='constant')
clf.fit(x_train_robust,y_train_robust)

train_yhat = clf.predict(x_train_robust)
train_f1 = f1_score(y_train_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_robust)
test_f1 = f1_score(y_test_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_robust,train_yhat))
print('Test Score\n',classification_report(y_test_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       0.98      0.96      0.97     25830
           1       0.81      0.15      0.25     25165
           2       1.00      1.00      1.00     26245
           3       0.52      0.95      0.67     25744

    accuracy                           0.77    102984
   macro avg       0.83      0.76      0.72    102984
weighted avg       0.83      0.77      0.73    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.89      0.94     11292
           1       0.86      0.16      0.27      4496
           2       0.99      1.00      1.00      3856
           3       0.00      1.00      0.01        16

    accuracy                           0.75     19660
   macro avg       0.71      0.76      0.56     19660
weighted avg       0.97      0.75      0.80     19660

AVG F1-Score Train: 0.7230310256202703
AVG F1-Score Test: 0.5553946416200737


# Parameter Grid for std

In [12]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_std,y_train_std)
    answer = clf.predict(x_test_std)
    f1 = f1_score(y_test_std,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

F1: 0.74016
Grid: {'activation': 'tanh', 'learning_rate': 'invscaling'}


In [14]:
clf = MLPClassifier(activation='tanh',learning_rate='invscaling')
clf.fit(x_train_std,y_train_std)

train_yhat = clf.predict(x_train_std)
train_f1 = f1_score(y_train_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_std)
test_f1 = f1_score(y_test_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_std,train_yhat))
print('Test Score\n',classification_report(y_test_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       0.81      0.99      0.89     25830
           1       0.56      1.00      0.72     25165
           2       1.00      1.00      1.00     26245
           3       0.37      0.00      0.00     25744

    accuracy                           0.75    102984
   macro avg       0.68      0.75      0.65    102984
weighted avg       0.68      0.75      0.65    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     11292
           1       0.96      0.99      0.98      4496
           2       0.99      1.00      0.99      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.99     19660
   macro avg       0.74      0.74      0.74     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.6518364775980865
AVG F1-Score Test: 0.7399979487995794


In [21]:
cross_std = cross_val_score(MLPClassifier(activation='tanh',learning_rate='constant'),x_train_robust,y_train_robust,cv=5)
print(round(np.mean(cross_std),4))

0.7742


In [22]:
cross_std = cross_val_score(MLPClassifier(activation='tanh',learning_rate='invscaling'),x_train_robust,y_train_robust,cv=5)
print(round(np.mean(cross_std),4))

0.7849


### Gridsearch for F class

In [6]:
x_train_f_std = x_train.copy()
y_train_f_std = y_train.copy()

x_test_f_std = x_test.copy()
y_test_f_std = y_test.copy()

x_train_f_std = x_train_f_std.loc[:,['pkts_received','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_f_std = y_train_f_std.loc[:,'Action']
x_test_f_std = x_test_f_std.loc[:,['pkts_received','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_f_std = y_test_f_std.loc[:,'Action']


x_train_f_robust = x_train.copy()
y_train_f_robust = y_train.copy()

x_test_f_robust = x_test.copy()
y_test_f_robust = y_test.copy()

x_train_f_robust = x_train_f_robust.loc[:,['pkts_received','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_f_robust = y_train_f_robust.loc[:,'Action']
x_test_f_robust = x_test_f_robust.loc[:,['pkts_received','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_f_robust = y_test_f_robust.loc[:,'Action']

In [7]:
scaler = StandardScaler()
robust = RobustScaler()
x_train_f_std = scaler.fit_transform(x_train_f_std)
x_test_f_std = scaler.fit_transform(x_test_f_std)

x_train_f_robust = robust.fit_transform(x_train_f_robust)
x_test_f_robust = robust.fit_transform(x_test_f_robust)

#### std

In [8]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_f_std,y_train_f_std)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_f_std)
print(classification_report(y_test_f_std, answer))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimize

{'activation': 'identity', 'learning_rate': 'constant'}
MLPClassifier(activation='identity')
              precision    recall  f1-score   support

           0       1.00      0.28      0.44     11292
           1       0.00      0.00      0.00      4496
           2       0.23      1.00      0.38      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.36     19660
   macro avg       0.31      0.32      0.20     19660
weighted avg       0.62      0.36      0.32     19660



/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [9]:
clf = MLPClassifier(activation='identity',learning_rate='constant')
clf.fit(x_train_f_std,y_train_f_std)

train_yhat = clf.predict(x_train_f_std)
train_f1 = f1_score(y_train_f_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_f_std)
test_f1 = f1_score(y_test_f_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_f_std,train_yhat))
print('Test Score\n',classification_report(y_test_f_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     25830
           1       0.00      0.00      0.00     25165
           2       0.37      1.00      0.54     26245
           3       0.96      0.23      0.37     25744

    accuracy                           0.56    102984
   macro avg       0.58      0.55      0.48    102984
weighted avg       0.58      0.56      0.48    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.28      0.44     11292
           1       0.00      0.00      0.00      4496
           2       0.23      1.00      0.38      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.36     19660
   macro avg       0.31      0.32      0.20     19660
weighted avg       0.62      0.36      0.32     19660

AVG F1-Score Train: 0.47610752426921543
AVG F1-Score Test: 0.2037756596482237


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

#### robust

In [10]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_f_robust,y_train_f_robust)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_f_robust)
print(classification_report(y_test_f_robust, answer))

{'activation': 'relu', 'learning_rate': 'invscaling'}
MLPClassifier(learning_rate='invscaling')
              precision    recall  f1-score   support

           0       1.00      0.51      0.68     11292
           1       0.47      1.00      0.64      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.52     19660
   macro avg       0.37      0.38      0.33     19660
weighted avg       0.68      0.52      0.53     19660



In [11]:
clf = MLPClassifier(activation='relu',learning_rate='invscaling')
clf.fit(x_train_f_robust,y_train_f_robust)

train_yhat = clf.predict(x_train_f_robust)
train_f1 = f1_score(y_train_f_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_f_robust)
test_f1 = f1_score(y_test_f_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_f_robust,train_yhat))
print('Test Score\n',classification_report(y_test_f_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25830
           1       0.35      1.00      0.52     25165
           2       0.00      0.00      0.00     26245
           3       1.00      0.24      0.38     25744

    accuracy                           0.55    102984
   macro avg       0.59      0.56      0.48    102984
weighted avg       0.59      0.55      0.47    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.50      0.66     11292
           1       0.47      1.00      0.64      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.51     19660
   macro avg       0.37      0.37      0.32     19660
weighted avg       0.68      0.51      0.53     19660

AVG F1-Score Train: 0.476923752986212
AVG F1-Score Test: 0.32496239826645357


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

### Gridserch RF

In [12]:
x_train_rf_std = x_train.copy()
y_train_rf_std = y_train.copy()

x_test_rf_std = x_test.copy()
y_test_rf_std = y_test.copy()

x_train_rf_std = x_train_rf_std.loc[:,['Source Port', 'NAT Source Port',
       'NAT Destination Port', 'Bytes', 'Bytes Sent', 'Bytes Received',
       'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']]
y_train_rf_std = y_train_rf_std.loc[:,'Action']
x_test_rf_std = x_test_rf_std.loc[:,['Source Port', 'NAT Source Port',
       'NAT Destination Port', 'Bytes', 'Bytes Sent', 'Bytes Received',
       'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']]
y_test_rf_std = y_test_rf_std.loc[:,'Action']


x_train_rf_robust = x_train.copy()
y_train_rf_robust = y_train.copy()

x_test_rf_robust = x_test.copy()
y_test_rf_robust = y_test.copy()

x_train_rf_robust = x_train_rf_robust.loc[:,['Source Port', 'NAT Source Port',
       'NAT Destination Port', 'Bytes', 'Bytes Sent', 'Bytes Received',
       'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']]
y_train_rf_robust = y_train_rf_robust.loc[:,'Action']
x_test_rf_robust = x_test_rf_robust.loc[:,['Source Port', 'NAT Source Port',
       'NAT Destination Port', 'Bytes', 'Bytes Sent', 'Bytes Received',
       'Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']]
y_test_rf_robust = y_test_rf_robust.loc[:,'Action']

In [13]:
scaler = StandardScaler()
robust = RobustScaler()
x_train_rf_std = scaler.fit_transform(x_train_rf_std.loc[:,['Bytes','Bytes Sent', 'Bytes Received','Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']])
x_test_rf_std = scaler.fit_transform(x_test_rf_std.loc[:,['Bytes','Bytes Sent', 'Bytes Received','Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']])

x_train_rf_robust = robust.fit_transform(x_train_rf_robust.loc[:,['Bytes','Bytes Sent', 'Bytes Received','Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']])
x_test_rf_robust = robust.fit_transform(x_test_rf_robust.loc[:,['Bytes','Bytes Sent', 'Bytes Received','Packets', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received']])

#### std

In [14]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_rf_std,y_train_rf_std)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_rf_std)
print(classification_report(y_test_rf_std, answer))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimize

{'activation': 'relu', 'learning_rate': 'adaptive'}
MLPClassifier(learning_rate='adaptive')
              precision    recall  f1-score   support

           0       1.00      0.28      0.44     11292
           1       0.00      0.00      0.00      4496
           2       0.23      1.00      0.38      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.36     19660
   macro avg       0.31      0.32      0.20     19660
weighted avg       0.62      0.36      0.32     19660



/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [15]:
clf = MLPClassifier(activation='relu',learning_rate='adaptive')
clf.fit(x_train_rf_std,y_train_rf_std)

train_yhat = clf.predict(x_train_rf_std)
train_f1 = f1_score(y_train_rf_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_rf_std)
test_f1 = f1_score(y_test_rf_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_rf_std,train_yhat))
print('Test Score\n',classification_report(y_test_rf_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     25830
           1       0.35      1.00      0.52     25165
           2       0.00      0.00      0.00     26245
           3       0.95      0.23      0.37     25744

    accuracy                           0.55    102984
   macro avg       0.58      0.55      0.47    102984
weighted avg       0.57      0.55      0.47    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.27      0.43     11292
           1       0.00      0.00      0.00      4496
           2       0.23      1.00      0.38      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.35     19660
   macro avg       0.31      0.32      0.20     19660
weighted avg       0.62      0.35      0.32     19660

AVG F1-Score Train: 0.4716102870449804
AVG F1-Score Test: 0.20087161995326003


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

#### robust

In [16]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_rf_robust,y_train_rf_robust)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_rf_robust)
print(classification_report(y_test_rf_robust, answer))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimize

{'activation': 'relu', 'learning_rate': 'adaptive'}
MLPClassifier(learning_rate='adaptive')
              precision    recall  f1-score   support

           0       1.00      0.50      0.66     11292
           1       0.46      1.00      0.63      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.51     19660
   macro avg       0.37      0.37      0.32     19660
weighted avg       0.68      0.51      0.53     19660



In [17]:
clf = MLPClassifier(activation='relu',learning_rate='adaptive')
clf.fit(x_train_rf_robust,y_train_rf_robust)

train_yhat = clf.predict(x_train_rf_robust)
train_f1 = f1_score(y_train_rf_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_rf_robust)
test_f1 = f1_score(y_test_rf_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_rf_robust,train_yhat))
print('Test Score\n',classification_report(y_test_rf_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25830
           1       0.84      0.58      0.68     25165
           2       0.78      1.00      0.88     26245
           3       0.84      0.86      0.85     25744

    accuracy                           0.86    102984
   macro avg       0.87      0.86      0.85    102984
weighted avg       0.87      0.86      0.85    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.45      0.62     11292
           1       0.46      0.99      0.63      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.19      0.00        16

    accuracy                           0.48     19660
   macro avg       0.37      0.41      0.31     19660
weighted avg       0.68      0.48      0.50     19660

AVG F1-Score Train: 0.8533073806250451
AVG F1-Score Test: 0.31235301257186954


### Gridserch NAD

In [18]:
x_train_nad_std = x_train.copy()
y_train_nad_std = y_train.copy()

x_test_nad_std = x_test.copy()
y_test_nad_std = y_test.copy()

x_train_nad_std = x_train_nad_std.loc[:,[ 'Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']]
y_train_nad_std = y_train_nad_std.loc[:,'Action']
x_test_nad_std = x_test_nad_std.loc[:,[ 'Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']]
y_test_nad_std = y_test_nad_std.loc[:,'Action']


x_train_nad_robust = x_train.copy()
y_train_nad_robust = y_train.copy()

x_test_nad_robust = x_test.copy()
y_test_nad_robust = y_test.copy()

x_train_nad_robust = x_train_nad_robust.loc[:,[ 'Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']]
y_train_nad_robust = y_train_nad_robust.loc[:,'Action']
x_test_nad_robust = x_test_nad_robust.loc[:,[ 'Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']]
y_test_nad_robust = y_test_nad_robust.loc[:,'Action']

In [19]:
scaler = StandardScaler()
robust = RobustScaler()
x_train_nad_std = scaler.fit_transform(x_train_nad_std.loc[ :,['Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']])
x_test_nad_std = scaler.fit_transform(x_test_nad_std.loc[ :,['Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']])

x_train_nad_robust = robust.fit_transform(x_train_nad_robust.loc[ :,['Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']])
x_test_nad_robust = robust.fit_transform(x_test_nad_robust.loc[ :,['Bytes Sent', 'Bytes Received', 'pkts_sent', 'pkts_received']])

#### std

In [20]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_nad_std,y_train_nad_std)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_nad_std)
print(classification_report(y_test_nad_std, answer))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimize

{'activation': 'tanh', 'learning_rate': 'invscaling'}
MLPClassifier(activation='tanh', learning_rate='invscaling')
              precision    recall  f1-score   support

           0       1.00      0.08      0.15     11292
           1       0.00      0.00      0.00      4496
           2       0.21      1.00      0.34      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.24     19660
   macro avg       0.30      0.27      0.12     19660
weighted avg       0.62      0.24      0.16     19660



/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [21]:
clf = MLPClassifier(activation='tanh',learning_rate='invscaling')
clf.fit(x_train_nad_std,y_train_nad_std)

train_yhat = clf.predict(x_train_nad_std)
train_f1 = f1_score(y_train_nad_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_nad_std)
test_f1 = f1_score(y_test_nad_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_nad_std,train_yhat))
print('Test Score\n',classification_report(y_test_nad_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Score
               precision    recall  f1-score   support

           0       0.81      0.91      0.86     25830
           1       0.00      0.00      0.00     25165
           2       0.36      1.00      0.53     26245
           3       0.21      0.01      0.02     25744

    accuracy                           0.49    102984
   macro avg       0.35      0.48      0.35    102984
weighted avg       0.35      0.49      0.36    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.09      0.16     11292
           1       0.00      0.00      0.00      4496
           2       0.21      1.00      0.34      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.25     19660
   macro avg       0.30      0.27      0.13     19660
weighted avg       0.62      0.25      0.16     19660

AVG F1-Score Train: 0.35306861280238755
AVG F1-Score Test: 0.12578807636125539


/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

#### robust

In [22]:
clf_best = GridSearchCV(clf, parameters, cv=5)
#knn_best.fit(X,y_train)
clf_best.fit(x_train_nad_robust,y_train_nad_robust)
print(clf_best.best_params_)
print(clf_best.best_estimator_)
answer = clf_best.predict(x_test_nad_robust)
print(classification_report(y_test_nad_robust, answer))

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimize

{'activation': 'relu', 'learning_rate': 'adaptive'}
MLPClassifier(learning_rate='adaptive')
              precision    recall  f1-score   support

           0       1.00      0.50      0.67     11292
           1       0.47      1.00      0.64      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.51     19660
   macro avg       0.37      0.37      0.33     19660
weighted avg       0.68      0.51      0.53     19660



/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
clf = MLPClassifier(activation='relu',learning_rate='adaptive')
clf.fit(x_train_nad_robust,y_train_nad_robust)

train_yhat = clf.predict(x_train_nad_robust)
train_f1 = f1_score(y_train_nad_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_nad_robust)
test_f1 = f1_score(y_test_nad_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_nad_robust,train_yhat))
print('Test Score\n',classification_report(y_test_nad_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       1.00      0.95      0.98     25830
           1       0.84      0.57      0.68     25165
           2       0.76      1.00      0.86     26245
           3       0.83      0.86      0.85     25744

    accuracy                           0.85    102984
   macro avg       0.86      0.85      0.84    102984
weighted avg       0.86      0.85      0.84    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.53      0.69     11292
           1       0.49      0.99      0.65      4496
           2       0.00      0.00      0.00      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.53     19660
   macro avg       0.37      0.38      0.34     19660
weighted avg       0.69      0.53      0.55     19660

AVG F1-Score Train: 0.8424495330555499
AVG F1-Score Test: 0.3366787867467589
